In [4]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import os
import re

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import dask
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm
import xarray as xr

In [5]:
energy = "solar"

In [7]:
file_path = f"../data/global-energy-monitor/Global-{energy.capitalize()}-Power-Tracker-December-2023.xlsx"
largescale = pd.read_excel(file_path, sheet_name=1)
mediumscale = pd.read_excel(file_path, sheet_name=2)
if energy == "solar":
    smallscale = pd.read_excel(file_path, sheet_name=3)
    combined_df = pd.concat(
        [largescale, mediumscale, smallscale], ignore_index=True
    )
else:
    combined_df = pd.concat([largescale, mediumscale], ignore_index=True)
combined_df = combined_df[combined_df["Status"] != "cancelled"]
combined_df = combined_df[combined_df["Status"] != "shelved"]
combined_df = combined_df[combined_df["Status"] != "retired"]
combined_df = combined_df[combined_df["Status"] != "mothballed"]

In [9]:
# sum Capacity (MW) by Technology Type
combined_df = combined_df.groupby("Technology Type").sum() 

In [10]:
combined_df

,Date Last Researched,Country,Project Name,Phase Name,Project Name in Local Language / Script,Other Name(s),Capacity (MW),Capacity Rating,Status,Start year,...,"Local area (taluk, county)","Major area (prefecture, district)",State/Province,Subregion,Region,GEM location ID,GEM phase ID,Other IDs (location),Other IDs (unit/phase),Wiki URL
Technology Type,,,,,,,,,,,,,,,,,,,,,
Assumed PV,2023/08/112023/08/152023/07/142023/07/142023/0...,AlgeriaAlgeriaEgyptEgyptLibyaMoroccoMoroccoMor...,Bir Rebaa Solar PV PlantVNG AG solar farmDEME-...,212121future phases121232212121234512111231212...,محطة وذرف للطاقة الشمسيةPlanta Solar Patacamay...,"Bengue-Bougou, BingebougouNobelsfontein Solar ...",159823.0,unknownunknownunknownunknownunknownunknownunkn...,constructionannouncedpre-constructionpre-const...,3102107.0,...,d'In Salah DistrictSuez Canal Economic Zonecer...,Tinghir ProvinceTinghir ProvinceAit Mellal Pro...,In SalahMediterranean SeaSuezDistrict of Al-Ju...,Northern AfricaNorthern AfricaNorthern AfricaN...,AfricaAfricaAfricaAfricaAfricaAfricaAfricaAfri...,L800058L827607L827626L827627L827652L827679L827...,G831103G831128G831154G831155G831183G831223G831...,WRI: WKS0061716WRI: WRI1026050WRI: WRI1026051W...,WKSL: 71888WKSL: 69771WKSL: 61716WKSL: 61052WK...,https://gem.wiki/Bir_Rebaa_Solar_PV_Planthttps...
PV,2022/08/162023/08/112022/08/162022/08/162023/0...,AlgeriaAlgeriaAlgeriaAlgeriaAlgeriaAlgeriaAlge...,Adrar Solar PlantAin Azel solar farmAin El Ibe...,PV 1PV 2112312121212345123451PV 1PV 2PV 3PV 1P...,محطة توليد الكهرباء بعين زادةمحطة عين الابل لل...,Ain Albel 1 solar plantAin Albel 2 solar plant...,2190192.3,MWp/dcunknownunknownunknownMWp/dcMWp/dcMWp/dcu...,operatingoperatingoperatingoperatingoperatingo...,47998865.0,...,Adrar DistrictAin ArnatAïn El Ibel DistrictAïn...,BB ArreidjBantaHauts plateaux eastOuarglaBiskr...,AdrarSétifDjelfaDjelfaM'SilaSaïdaAdrarBatnaOua...,Northern AfricaNorthern AfricaNorthern AfricaN...,AfricaAfricaAfricaAfricaAfricaAfricaAfricaAfri...,L800008L800010L800012L800012L800013L800014L800...,G800008G800010G800012G800013G800014G800015G800...,WRI: WKS0068905WRI: WKS0065285WRI: WKS0068916W...,WKSL: 68915WKSL: 68919WKSL: 65283WKSL: 72833WK...,https://gem.wiki/Adrar_Solar_Planthttps://gem....
Solar Thermal,2023/08/112023/07/072023/07/072023/07/052023/0...,AlgeriaEgyptEgyptEgyptMoroccoMoroccoMoroccoMor...,Hassi R'Mel ISCC power stationISCC Kuraymat Th...,Solar Thermalمحطة عين بني مطهر للطاقة الشمسية1...,محطة كهرباء حاسي الرملمحطة الكريمات الشمسيةمحط...,"Tilghemt ISCC, SPP1 Thermal, Hassi R'Mel Gas S...",21205.2,MWacMWacMWp/dcunknownMWacMWacMWacMWacMWacMWacM...,operatingoperatingoperatingpre-constructionope...,272193.0,...,El-SafWest Nilecercle d'Aïn Bni Mathar دائرة ع...,Hassi R'melJerada ProvinceMidelt ProvinceOuarz...,GizaEl BeheiraWest NileKébiliKébiliCentral Dis...,Northern AfricaNorthern AfricaNorthern AfricaN...,AfricaAfricaAfricaAfricaAfricaAfricaAfricaAfri...,L406051L811586L811587L827635L811624L800243L800...,G408032G814096G814097G831163G814163G800289G800...,"OpenCSP: 38OpenCSP: 39WRI: WKS0071080, OpenCSP...",WKSL: 71085WKSL: 71080WKSL: 61338WKSL: 63891WK...,https://gem.wiki/Hassi_R'Mel_ISCC_power_statio...
assumed PV,2023/02/072023/02/072023/01/062023/01/062023/0...,PolandPolandRussiaRussiaRussiaRussiaRussiaSlov...,Recz solar farmWierzbica solar farmKrasinskaya...,1211232,Красинская СЭСКурчалоевская СЭСЛаганская СЭСМа...,Ingka Recz solar farmPredgornaya solar farmSps...,3524.6,unknownunknownunknownunknownunknownunknownunkn...,pre-constructionpre-constructionannouncedpre-c...,86919.0,...,Choszczno CountyRadom CountyOrskNorth Ostrobot...,Trnava DistrictRegional State Administrative A...,West Pomeranian VoivodeshipMasovian Voivodeshi...,Eastern EuropeEastern EuropeEastern EuropeEast...,EuropeEuropeEuropeEuropeEuropeEuropeEuropeEuro...,L824064L824073L824268L824270L824271L824272L804...,G827134G827143G827370G827372G827373G827374G804...,WRI: WRI1062070,0,https://gem.wiki/Recz_solar_farmhttps://gem.wi...


In [11]:
2190192.3/21205.2

103.28562333767188